# Data preparation
Data - Information - Knowledge
While *data* is, what is stored and transported (for example strings, integers, pixels, etc.), *information* is a result of data interpretation when analysing and visualizing. But on top of data and information comes *knowledge* with comparing, remembering and learning.  

In the first part of this notebook we are going to take a look at data preparation using python, covering the criteria for data quality, data cleaning and data summarising, combining and reducing.
Even though an incredible amount of data is generated and used every day, high quality and up-to-date datasets are rare. Furthermore, pushing data around (converting to other formats, or data structure, etc) could rise in data loss, therefore always check the data before moving on to the next step. 

**Important**: *Just a little hint, all data we use in this course are focusing on Aachen. Since this is a German city, the data we are using is mainly in German. We do try to translate as often as possible and most of the processing is focussing on geolocated or statistical data however, it could be that there are German words coming up.*

## Data quality
Let us start with the criteria of data quality. These criteria are:

* Correctness/ Accuracy
* Reliability/ Credibility
* Consistency
* Completeness
* Comprehensibility
* No Redundancy
* Timeliness
* Accessibility 

So far so good, well having these criteria is one thing, applying them on a data set another. So let us take the example **Recyclinghöfe.json** (which is actually a geoJSON, as you will see) and check if it matches the criteria. 

When we go through the list, we will see, that the data file **Recyclinghöfe.json** was downloaded from the Aachen [Open data Portal](https://offenedaten.aachen.de/). 

**Reliability/ Credibility** - 
They do provide a few more information about this data set such as: *Source, Author, Person responsible, Last updated, Created*. However, it is not provided any information on how the data was created. For its purpose to be an example of data within this course, it is enough. However, depending on the purpose the data is meant to full fill, more information are maybe required.   

**No redundancy** - 
Well, if one check the plot of our read data file one may discover that there is some data occurring twice. You may have seen it already, it is the longitude and latitude information. This information is once in the geometry column as point definition and also in separated columns *lat* and *log*. This does not necessarily mean it is redundant since the differentiation in log and lat allows to know what is what, since within the geometry column this is actually not defined (common sense may tell you, the other option is not even close to Germany, but well sometimes no one cares about common sense). Anyway also we could talk about a redundancy here, it is not a troubling issue which needs to be solved for our purpose.

![redundancy](./img/redundancy.png)  

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Reading/ Parsing all necessary files
Let us start by reading the data file **Recyclinghöfe.json** using the Python package GeoPandas.

In [ ]:
recycling = gpd.read_file("data/Recyclinghöfe.json")

GeoPandas now has read the data from the geoJSON file with its geometric information and the data on the attributes and parsed them. The result is a table, a **GeoDataFrame**. We can inspect the table using the **.plot()** function offered by the GeoPandas GeoDataFrame.

In [ ]:
recycling.head()

In [ ]:
recycling.plot()

## Data cleaning
We have a few options to clean our data set:

* Ignore data set
* Manual insertion
* Global constant
* Location parameter
* Restricted location parameter
* Most probable value
* Time series and spatial data

Since only the fact, that we have two global constants, occurs in the **Recyclinghöfe.json** file, this is the only interesting aspect. However, the global constants used in our example file is not disturbing and actually does not require any cleaning since it has no direct impact on our purpose. But in case we would want to know, in which district of Aachen each of this recycling facilities is placed, then this information would be helpful.

Well, since we have the location given, and the data set only consists of 4 objects, we could go ahead and figure out in which district of Aachen each of this recycling facilities is placed. This is a manual task to do. One would search for the **ortsteil** by searching for the address and then manually enter the result in the geoJSON file.

## Data - Combining, summarizing, reducing 
We not go to cover all three, but focus on an example for combining data. 
Therefore, we need another example apart from **Recyclinghöfe.json** since this file does not enough data, or let us say interesting enough data to be an useful example. Instead of Recyclinghöfe.json we choose two other data files which are **einwohnerstatistik-31.12.2020.csv** and **StatistischeBezirkeAachen.shp**. 
So let us start with (again) reading the data files using Python GeoPandas. 

**Reading StatistischeBezirkeAachen.shp**

In [ ]:
districts = gpd.read_file("./data/aachen/StatistischeBezirkeAachen.shp")

In [ ]:
districts.head()

In [ ]:
districts.plot()

**Reading einwohnerstatistik-31.12.2020.csv**

In [ ]:
statistics = pd.read_csv("./data/aachen/einwohnerstatistik-31.12.2020.csv")
statistics.head()
statistics.plot()

### Investigating the information for commonalities
After reading the files using GeoPandas we can start checking them for information we need. But first, we require to clarify what we are going to do. Well we want to combine the two data set and in order to do this we require information based on which we can perform a combination - so let us check both files for data that allows us to combine them.
The CSV file is a plain table file, which means it does not include any geometric information. However both data file have one feature in common: the **id** of the area. Even though the columns names differ, they are the same entity though. Let's order the tables by the respective id and see whether they match.


In [ ]:
# resorting the statistic DataFrame
sorted_stats = statistics.sort_values(by='StatBezName')
sorted_stats.head()

In [ ]:
# resorting the districts GeoDataFrame
sorted_districts = districts.sort_values(by='ST_NAME')
sorted_districts.head()

Since we know now, that **id** is an attribute both tables actually have even though it is named differently. 
With this information we can add the information from the extended statistics DataFrame we have assigned in `statistics` to the GeoDataFrame. Here, every column of the `statistics` data set is added to the GeoDataFrame table. With tabular data, this concept is referred to as **merging** or **joining** data. 

An explanation on merging GeoDataFrames is provided here: https://geopandas.org/docs/user_guide/mergingdata.html 

In order to perform a **join** we need to make sure to define on WHAT we are going to join and how. In our case, joining the *statistics* DataFrame to the *districts* GeoDataFrame from the left.
```
bezirke <-- statistics
```

### Renaming columns 
First however, we have to rename the column name *Bez* from the *statistics* DataFrame to match the respective name *STATBEZ* column in our *districts* GeoDataFrame

In [ ]:
statistics = statistics.rename(columns = {'Bez' : 'STATBEZ'})
statistics.head(100)

### Perparing information
In a next step, we clean our data. As already talked about earlier in this notebook, clean data is essential for efficient and qualitative data processing, and therefore we need to get rid of unwanted elements such as *NaN*.

In [ ]:
# pd.to_numeric(statistics["STATBEZ"])
import numpy as np
statistics = statistics.replace(r'^\s*$', np.nan, regex=True)
index = statistics[statistics['STATBEZ'] == np.nan].index
#statistics.drop(34, inplace=True)
statistics.head(20)

Next we have an example for using NumPy dtype. Here, we define the column content for *STATBEZ* as type integer64. Integer, as we want them to be hole numbers not floating point numbers and 64 defines the size of date, here 64 bytes.

In [ ]:
statistics["STATBEZ"] = statistics["STATBEZ"].fillna(0)
statistics["STATBEZ"] = statistics["STATBEZ"].astype('int64')

statistics["STATBEZ"].tail()

### Merging information into one DataFrame
Finally, we get to the point where we can actually merge our two data sets. 
We do this by using the Python GeoPandas *merge* method (remember, a function is called a method if it is defined within a class).  

You may have seen the *tail* method, this in contrast to the *head* method gives us the last rows of a DataFrame table.

In [ ]:
districts_stats = districts.merge(statistics, on='STATBEZ', how='left')
# districts_stats = districts.set_index('STATBEZ').join(statistics.set_index('STATBEZ'))
districts_stats.tail(3)

In [ ]:
districts_stats = gpd.GeoDataFrame(districts_stats)
# type is used to get to know what python type or data type in python a specific entity is. Here for example we get to know that the variable districts_stats is a GeoDataFrame
type(districts_stats)

In [ ]:
districts_stats.head(5)

### Adding information
Next, we are going to create new information by combining existing ones. To be more precise, we are going to add the **population density**. We can do this because we already know the districts since we get their geometry as a polygon (check the GeoDataFrame geometry column) and since we just merged these two data sets we also have the number of people living in each district.

But first we need some visualization because this is just easier for humans to read. Looking at a nice pretty picture is so much more fun than text... or code. Therefore, let us create a plot visualization using Python matplotlib library.

In [ ]:
# set to min and max of data
vmin, vmax = 0, 133623

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(14,6))
# add a title and annotation
ax.set_title('Total population of Aachen', fontdict={'fontsize': '15', 'fontweight' : '3'})

ax = districts_stats.plot(column='Pers', cmap = 'YlGnBu', ax=ax, legend = True)

Now let us get the area size of each district. 

First, we slice the GeoDataFrame to get the geometry column content only. Now we have the polygons (represented by points) and we now want to get the area. Fortunately we do not need to calculate it ourselves, but Python GeoPanda provides us with the *area* method, and this it what we are going to use here.

In [ ]:
print(districts_stats[1:5]["geometry"].area)

Well, the area calculation seems to be successful, so we can move on and put all components in the formula to calculate the population density (here called *popdens*).

Additionally, since we not only want to calculate the population density, but we also want to save this information, we are going to add a new column to our GeoDataFrame by adding a *popdens* column.

In [ ]:
districts_stats['popdens'] = districts_stats["Pers"] / districts_stats["geometry"].area*1000

In [ ]:
districts_stats.head()

In a last and final step, we want to visualize the results again. Since visualization is one of the most important aspects in data processing, always try to find a way to meaningfully represent your data and results.

In [ ]:
# set to min and max of data
vmin, vmax = 0, 133623

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(14,6))
# add a title and annotation
ax.set_title('Population Density of Aachen in 1000 inhabitants per km^2', fontdict={'fontsize': '15', 'fontweight' : '3'})

ax = districts_stats.plot(column='popdens', cmap = 'YlGnBu', ax=ax, legend = True)